<a href="https://colab.research.google.com/github/ekusz/dw_matrix_car/blob/master/day4_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [5]:
cd /content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [12]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

##Feature Engineering

In [0]:
SUFFIX_CAT = '_cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values
  

In [14]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model,feats):
  x = df[feats].values
  y = df['price_value'].values


  scores = cross_val_score(model,x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

##DecisionTree

In [19]:
  run_model ( DecisionTreeRegressor(max_depth=5), cat_feats )

(-19695.13091100928, 148.72570644015937)

##Random Forest

In [21]:
  model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0 )
  run_model (model, cat_feats )

(-18718.657185256638, 64.5424578125788)

##XGboost

In [22]:
xgb_params = {
    'max_deph':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats )

[15:06:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:06:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-15897.787646158193, 145.21306181010542)

In [23]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(x, y)

imp = PermutationImportance(m, random_state=0).fit(x, y)
eli5.show_weights(imp, feature_names=cat_feats)

[15:10:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd_cat
0.1132 ± 0.0032,param_rok-produkcji_cat
0.1090 ± 0.0025,param_stan_cat
0.0619 ± 0.0025,param_skrzynia-biegów_cat
0.0568 ± 0.0016,param_faktura-vat_cat
0.0489 ± 0.0014,param_moc_cat
0.0273 ± 0.0007,param_marka-pojazdu_cat
0.0242 ± 0.0013,feature_kamera-cofania_cat
0.0212 ± 0.0008,param_typ_cat
0.0174 ± 0.0008,param_pojemność-skokowa_cat


In [26]:
len(cat_feats)

151

In [33]:
feats = ['param_napęd_cat', 'param_rok-produkcji_cat', 'param_stan_cat', 'param_skrzynia-biegów_cat', 'param_faktura-vat_cat', 'param_moc_cat', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 'param_typ_cat', 'param_pojemność-skokowa_cat', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat', 'param_model-pojazdu_cat', 'param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat', 'feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat', 'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats )

[15:32:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:32:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:32:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-16456.20558941591, 129.85000258897418)

In [34]:
  df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

  feats = ['param_napęd_cat', 'param_rok-produkcji', 'param_stan_cat', 'param_skrzynia-biegów_cat', 'param_faktura-vat_cat', 'param_moc_cat', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 'param_typ_cat', 'param_pojemność-skokowa_cat', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat', 'param_model-pojazdu_cat', 'param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat', 'feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat', 'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']

  run_model(xgb.XGBRegressor(**xgb_params), feats )

[15:34:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:34:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:34:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13428.042982968635, 78.92875100701907)

In [38]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = ['param_napęd_cat', 'param_rok-produkcji', 'param_stan_cat', 'param_skrzynia-biegów_cat', 'param_faktura-vat_cat', 'param_moc', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 'param_typ_cat', 'param_pojemność-skokowa_cat', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat', 'param_model-pojazdu_cat', 'param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat', 'feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat', 'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']
run_model(xgb.XGBRegressor(**xgb_params), feats )

[15:45:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:45:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:45:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11502.98318275872, 146.53566372172148)

In [42]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')) )

feats = ['param_napęd_cat', 'param_rok-produkcji', 'param_stan_cat', 'param_skrzynia-biegów_cat', 'param_faktura-vat_cat', 'param_moc', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 'param_typ_cat', 'param_pojemność-skokowa', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat', 'param_model-pojazdu_cat', 'param_wersja_cat','param_kod-silnika_cat','feature_system-start-stop_cat', 'feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat', 'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']
run_model(xgb.XGBRegressor(**xgb_params), feats )

[15:59:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:59:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:59:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11297.628203796621, 54.49266013359019)